# Compare modeled and remotely-sensed surface mass balance

Requires mass balance model outputs at each site from [PyGEM](https://github.com/PyGEM-Community/PyGEM) (Rounce et al., 2023), which can be downloaded form the Carnegie Mellon data repository.

The files downloaded for this work were:
- Monthly surface mass balance along glacier centerlines for 2000–2022: "binned", downloaded from [global_ERA5_2000_2022](https://cmu.app.box.com/s/rzk8aeasg40dd3p0xr3yngkc5c0m8kxt/folder/251139952066)
- Calibrated model parameters (degree-day factors of snow and temperature biases): "{RGI ID}_modelprms_dict.json" files downloaded from [pygem_datasets > Calibration](https://cmu.app.box.com/s/p8aiby5s9f3n6ycgmhknbgo4htk3pn9j/folder/298954564072)

All files were placed in a folder called "Rounce_et_al_2023", defined with the `model_path` variable below. 

In [ ]:
import os
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from scipy.stats import median_abs_deviation as MAD
from tqdm.auto import tqdm
import glob
import geopandas as gpd

In [ ]:
# Define paths for inputs and outputs
scm_path = '/Volumes/LaCie/raineyaberle/Research/PhD/snow_cover_mapping/'
model_path = os.path.join(scm_path, 'Rounce_et_al_2023')
out_path = os.path.join(scm_path, 'analysis')
code_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/glacier-snow-cover-analysis'

# Load glacier boundaries for RGI IDs
aois_fn = os.path.join(scm_path, 'analysis', 'AOIs.gpkg')
aois = gpd.read_file(aois_fn)

## 1. Monthly snowline altitudes

### Remotely-sensed SLAs

In [ ]:
slas_obs_fn = os.path.join(out_path, 'monthly_SLAs_observed.nc')
if not os.path.exists(slas_obs_fn):
    # Initialize a list to store DataFrames
    slas_obs_list = []
    
    # Iterate over RGI IDs
    for rgi_id in tqdm(sorted(aois['RGIId'].drop_duplicates().values)):
        scs_fn = os.path.join(scm_path, 'study-sites', rgi_id, f'{rgi_id}_snow_cover_stats.csv')
        scs = pd.read_csv(scs_fn)
        scs['datetime'] = pd.to_datetime(scs['datetime'], format='mixed')
        scs['Year'] = scs['datetime'].dt.year
        scs['Month'] = scs['datetime'].dt.month
        scs['Day'] = scs['datetime'].dt.day
        
        # Filter data to within one week of the first of each month
        scs_filtered = scs[(scs['Day'] >= 25) | (scs['Day'] <= 7)]
        
        # Grab monthly snowline altitude
        Imonths = []
        dates = []
        for year, month in scs_filtered[['Year', 'Month']].drop_duplicates().values:
            first_of_month = pd.Timestamp(year=year, month=month, day=1)
            
            # Identify closest observation to this date
            scs_filtered.loc[:, 'diff'] = np.abs(scs_filtered.loc[:, 'datetime'] - first_of_month)
            Imonths.append(scs_filtered['diff'].idxmin())
            dates.append(first_of_month)
        
        scs_monthly = scs.iloc[Imonths].reset_index(drop=True)
        scs_monthly['Date'] = dates
        scs_monthly['RGIId'] = rgi_id
        slas_obs_list.append(scs_monthly[['RGIId', 'Date', 'SLA_from_AAR_m']])
    
    # Combine all DataFrames
    slas_obs = pd.concat(slas_obs_list)
    slas_obs.rename(columns={'SLA_from_AAR_m': 'SLA_obs_m'}, inplace=True)
    
    # Create xarray Dataset
    slas_obs_pivot = slas_obs.pivot(index='Date', columns='RGIId', values='SLA_obs_m')
    slas_obs_pivot = slas_obs_pivot.sort_index()
    
    # Convert to xarray Dataset
    slas_obs_xr = xr.Dataset(
        {"SLA_obs_m": (['time', 'RGIId'], slas_obs_pivot.values)},
        coords={"time": slas_obs_pivot.index.values,
                "RGIId": slas_obs_pivot.columns.values}
    )
    
    # Save to NetCDF file
    slas_obs_xr.to_netcdf(slas_obs_fn)
    print('Remotely-sensed monthly SLAs saved to file:', slas_obs_fn)

else:
    # Load from file
    slas_obs_xr = xr.load_dataset(slas_obs_fn)
    print('Remotely-sensed monthly SLAs loaded from file.')

slas_obs_xr


### Modeled SLAs and SMB at observed SLAs

In [ ]:
# Check if file already exists
slas_mod_fn = os.path.join(out_path, 'monthly_SLAs_modeled.nc')
if not os.path.exists(slas_mod_fn):
    
    # Initialize a list to store DataFrames
    slas_mod_list = []
    
    # Iterate over sites
    for rgi_id in tqdm(aois['RGIId'].drop_duplicates().values):
        # Load modeled monthly SMB
        smb_fn = glob.glob(os.path.join(model_path, 'glac_SMB_binned', f"{rgi_id.split('RGI60-0')[1]}*.nc"))[0]
        smb = xr.open_dataset(smb_fn)
        # calculate cumulative SMB
        def water_year(date):
            if date.month >= 10:
                return date.year
            else:
                return date.year - 1
        smb = smb.assign_coords({'water_year': (['time'], [water_year(t) for t in smb.time.values])})
        smb['bin_massbalclim_monthly_cumsum'] = smb['bin_massbalclim_monthly'].groupby('water_year').cumsum()
        smb['time'] = smb.time.values.astype('datetime64[D]')
        h = smb['bin_surface_h_initial'].data.ravel()
        
        # Interpolate modeled SLA as where SMB = 0 and SMB at the observed SLA
        slas = np.nan * np.zeros(len(smb.time.data))
        smb_at_slas = np.nan * np.zeros(len(smb.time.data))
        for j, t in enumerate(smb.time.data):
            smb_time = smb.sel(time=t)['bin_massbalclim_monthly_cumsum'].data[0]
            # when SMB <= 0 everywhere, set SLA to maximum glacier elevation
            if np.all(smb_time <= 0):
                slas[j] = np.max(h)
            # when SMB >= 0 everywhere, set SLA to minimum glacier elevation
            elif np.all(smb_time >= 0):
                slas[j] = np.min(h)
            # otherwise, linearly interpolate SLA
            else:
                sorted_indices = np.argsort(h)
                slas[j] = np.interp(0, smb_time[sorted_indices], h[sorted_indices])
            # interpolate the modeled SMB at the observed SLA
            sla_obs = slas_obs.loc[(slas_obs['RGIId']==rgi_id) & (slas_obs['Date']==t), 'SLA_obs_m']
            if len(sla_obs) > 0:
                smb_at_slas[j] = np.interp(sla_obs.values[0], h, smb_time)

        # Save results in dataframe
        df = pd.DataFrame({'RGIId': [rgi_id]*len(smb.time.data),
                            'Date': smb.time.data,
                            'SLA_mod_m': slas,
                            'SMB_at_SLA_obs_mwe': smb_at_slas})
        # concatenate to dataframe list
        slas_mod_list.append(df)
        
    # Combine all DataFrames
    slas_mod = pd.concat(slas_mod_list)
    
    # Create xarray Dataset
    slas_mod_pivot = slas_mod.pivot(index='Date', columns='RGIId', values=['SLA_mod_m', 'SMB_at_SLA_obs_mwe'])
    slas_mod_pivot = slas_mod_pivot.sort_index()
    
    # Convert to xarray Dataset
    slas_mod_xr = xr.Dataset(
        {"SLA_mod_m": (['time', 'RGIId'], slas_mod_pivot['SLA_mod_m'].values),
         "SMB_at_SLA_obs_mwe":(['time', 'RGIId'], slas_mod_pivot['SMB_at_SLA_obs_mwe'].values)},
        coords={"time": slas_mod_pivot.index.values,
                "RGIId": slas_mod_pivot.columns.levels[1].values}
    )
    
    # Save to NetCDF file
    slas_mod_xr.to_netcdf(slas_mod_fn)
    print('Modeled monthly SLAs and snowline SMB saved to file:', slas_mod_fn)
    
else:
    # Load from file
    slas_mod_xr = xr.load_dataset(slas_mod_fn)
    print('Modeled monthly SLAs loaded from file.')

slas_mod_xr

### Merge

In [ ]:
# Define output file
slas_merged_fn = os.path.join(out_path, 'monthly_SLAs_observed_modeled.nc')
if not os.path.exists(slas_merged_fn):

    # Merge modeled and remotely-sensed SLAs and modeled SMB at observed snowline
    slas_merged = xr.merge([slas_obs_xr, slas_mod_xr])
    
    # Remove 2000-2012 (no observed values) and 2023 (no modeled values)
    slas_merged.sel(time=slice("2013-01-01","2023-01-01"))
    
    # Remove observations outside May - November (no observed values)
    def filter_month_range(month):
        return (month >= 5) & (month <= 10)
    slas_merged = slas_merged.sel(time=filter_month_range(slas_merged['time.month']))
    
    # Save results
    slas_merged.to_netcdf(slas_merged_fn)
    print('Merged monthly SLAs saved to file:', slas_merged_fn)

else:
    slas_merged = xr.load_dataset(slas_merged_fn)
    print('Merged monthly SLAs loaded from file.')

slas_merged['SLA_mod-obs_m'] = slas_merged['SLA_mod_m'] - slas_merged['SLA_obs_m']

# Plot
fig, ax = plt.subplots(figsize=(6,5))
ax.hist(slas_merged['SLA_mod-obs_m'].values.ravel(), bins=50)
ax.set_xlabel('SLA$_{mod}$ - SLA$_{obs}$ [m]')
ax.set_ylabel('Counts')
plt.show()


print('\nDifference stats:')
print(f'Mean diff = {np.nanmean((slas_merged["SLA_mod-obs_m"]).values)} m')
print(f'Std. diff = {np.nanstd((slas_merged["SLA_mod-obs_m"]).values)} m')
print(f'Median diff = {np.nanmedian((slas_merged["SLA_mod-obs_m"]).values)} m')
print(f'MAD diff = {MAD((slas_merged["SLA_mod-obs_m"]).values.ravel(), nan_policy="omit")} m')

## 2. ELAs

In [ ]:
slas_elas_merged_fn = os.path.join(out_path, 'monthly_SLAs_annual_ELAs_observed_modeled.nc')

if not os.path.exists(slas_elas_merged_fn):
    # Make a copy of the SLAs
    slas_elas_merged = slas_merged.copy()
    
    # Identify maximum annual observed SLAs
    slas_elas_merged['ELA_obs_m'] = slas_elas_merged['SLA_obs_m'].groupby(['time.year']).max()
    
    # Identify maximum annual modeled SLAs 
    slas_elas_merged['ELA_mod_m'] = slas_elas_merged['SLA_mod_m'].groupby(['time.year']).max()
    
    # Save to NetCDF file
    slas_elas_merged.to_netcdf(slas_elas_merged_fn)
    print('Merged SLAs and ELAs saved to file:', slas_elas_merged_fn)

else:
    slas_elas_merged = xr.load_dataset(slas_elas_merged_fn)
    print('Merged SLAs and ELAs loaded from file.')

slas_elas_merged

## 3. Degree-day factors of snow

### Modeled

In [ ]:
# Check if already exists in file
fsnow_mod_fn = os.path.join(scm_path, 'analysis', 'fsnow_modeled.csv')
if not os.path.exists(fsnow_mod_fn):
    print('Compiling modeled melt factors of snow')
    # Initialize dataframe
    fsnow_mod = pd.DataFrame()
    # Iterate over RGI IDs
    for rgi_id in tqdm(slas_mod['RGIId'].drop_duplicates().values):
        # Load model parameters
        modelprm_fn = os.path.join(model_path, 'modelprms', f"{rgi_id.replace('RGI60-0','')}-modelprms_dict.pkl")
        modelprm = pd.read_pickle(modelprm_fn)
        # Take the median of MCMC fsnow results (not much different than the mean)
        ddfsnow_mcmc = np.array(modelprm['MCMC']['ddfsnow']['chain_0'])
        df = pd.DataFrame({"RGIId": [rgi_id],
                           "fsnow_mod_m/C/d": [np.median(ddfsnow_mcmc)]})
        # Concatenate df to full dataframe
        fsnow_mod = pd.concat([fsnow_mod, df])
    # Save to file
    fsnow_mod.reset_index(drop=True, inplace=True)
    fsnow_mod.to_csv(fsnow_mod_fn, index=False)
    print('Compiled melt factors of snow saved to file:', fsnow_mod_fn)

else:
    fsnow_mod = pd.read_csv(fsnow_mod_fn)
    print('Compiled melt factors of snow loaded from file.')

plt.hist(fsnow_mod['fsnow_mod_m/C/d'] * 1e3, bins=100)
plt.xlabel('Melt factor of snow [mm $^{\circ}$C$^{-1}$ d$^{-1}$]')
plt.ylabel('Counts')
plt.show()


### Observed

Adjust the modeled degree-day factors of snow ($f_{snow}$) using the modeled SMB and cumulative PDDs from ERA5 downscaled to the snowline.

$SMB(x,t) = Accumulation - Melt = \Sigma Snowfall(x,t) - \sum_{t_{melt}}^t PDD(x,t) \cdot \Delta t \cdot f_{snow}$

where $t_{melt}$ is the start of the melt season and $\Delta t$ is $t-t_{melt}$. 

At the snowline, SMB = 0. Rearranging:

$f_{snow}(x,t) = \frac{\Sigma Snowfall(x,t)}{\sum_{t_{melt}}^t PDD(x,t) \cdot \Delta t} $

If SMB = 10 m at the snowline on day 100 of the melt season and the cumulative PDD are 100 $^{\circ}C$, this means that the model underestimated melt by 10 m / (100 $^{\circ}C \cdot$ 100 days) = 0.001 m/C/d. If the modeled melt factor of snow is 2 m/C/d, adjust the fsnow to 2.001 m/C/d. 

In [ ]:
fsnow_merged_fn = os.path.join(out_path, 'fsnow_observed_modeled.csv')
if not os.path.exists(fsnow_merged_fn):

    # Intialize results for all sites
    fsnow_merged = pd.DataFrame()
    
    # Iterate over sites
    for rgi_id in tqdm(aois['RGIId'].drop_duplicates().values):
        if type(rgi_id) != str:
            continue
        if 'RGI' not in rgi_id:
            continue
            
        ### Load input data
        # Get modeled fsnow
        fsnow_mod_site = fsnow_mod.loc[fsnow_mod['RGIId']==rgi_id, 'fsnow_mod_m/C/d'].values[0]
        # Load ERA-Land data
        era_fn = os.path.join(scm_path, 'study-sites', rgi_id, 'ERA', f"{rgi_id}_ERA5-Land_daily_means.csv")
        era_df = pd.read_csv(era_fn)
        era_df['Date'] = pd.to_datetime(era_df['Date'])
        # Load model temperature bias
        modelprm_fn = os.path.join(model_path, 'modelprms', f"{rgi_id.replace('RGI60-0','')}-modelprms_dict.pkl")
        modelprm = pd.read_pickle(modelprm_fn)
        tbias = np.median(modelprm['MCMC']['tbias']['chain_0'])
        # Load centerline elevation profile
        smb_fn = glob.glob(os.path.join(model_path, 'glac_SMB_binned', f"{rgi_id.split('RGI60-0')[1]}*.nc"))[0]
        smb = xr.open_dataset(smb_fn).squeeze()
        h = smb.bin_surface_h_initial.values.ravel()
        # Grab snowlines for site
        slas_site = slas_merged.sel(RGIId=rgi_id)
        # Don't include dates after September
        def filter_month_range(month):
            return month < 9
        slas_site = slas_site.sel(time=filter_month_range(slas_site['time.month']))
        
        ### Downscale air temperatures to glacier surface using lapse rates, calculate PDDs
        era_ds = xr.Dataset(
            coords={'h': h, 'time': era_df['Date']},
            data_vars={'temp_C': (['time'], era_df['mean_temperature_2m_C'].values),
                    'lapse_rate': (['time'], era_df['lapse_rate_C/m'])})
        # implement temperature bias
        era_ds['temp_C'] -= tbias
        era_ds['h_diff'] = era_df['ERA5_height_mean_m'].values[0] - era_ds['h']
        era_ds['temp_downscaled_C'] = era_ds['temp_C'] - era_ds['lapse_rate'] * era_ds['h_diff']
        era_ds['PDD'] = xr.where(era_ds['temp_downscaled_C'] > 0, era_ds['temp_downscaled_C'], 0)
        era_ds['PDD_cumsum'] = era_ds['PDD'].groupby('time.year').cumsum()
        
        ### Identify the melt season start date (first PDD > 0) for each elevation
        def find_first_positive(group):
            # Mask PDD = 0
            mask = group > 0
            # Find the first index where PDD > 0
            first_index = mask.argmax(dim="time")
            # Check if no positive PDD exists for the group
            no_positive = ~mask.any(dim="time")
            # Grab the corresponding time values
            time_values = group["time"].isel(time=first_index)
            # Replace invalid times with NaT for no_positive cases
            time_values = time_values.where(~no_positive, np.datetime64("NaT"))
            return time_values
        # Apply the function to each year and elevation group
        era_ds['melt_season_start_date'] = (era_ds["PDD_cumsum"]
                                            .groupby("time.year")
                                            .map(find_first_positive))
        
        ### Interpolate cumulative PDDs at SLAs
        pdds_slas = np.array([float(era_ds.sel(time=date, h=sla, method='nearest')['PDD_cumsum'].values) 
                              for (date,sla) in list(zip(slas_site.time.values, slas_site['SLA_obs_m'].values))])

        ### Interpolate melt season start dates at SLAs
        melt_start_dates = np.array([era_ds.sel(year=year, h=sla, method='nearest')['melt_season_start_date'].values
                                     for (year,sla) in list(zip(slas_site['time.year'].values, slas_site['SLA_obs_m'].values))])

        # Compile in dataframe
        df = pd.DataFrame({'Date': slas_site.time.values,
                           'Year': slas_site['time.year'].values,
                           'SMB_at_SLA_obs_mwe': slas_site['SMB_at_SLA_obs_mwe'].values,
                           'PDD_cumsum_at_SLA_C': pdds_slas,
                           'melt_season_start_date': melt_start_dates})
        df['RGIId'] = rgi_id
        df.dropna(inplace=True)
        df = df.loc[df['PDD_cumsum_at_SLA_C'] > 0] # remove rows with 0 PDDs (to avoid dividing by 0)
        df['days_since_melt_season_start_date'] = ((df['Date'] - df['melt_season_start_date']) / np.timedelta64(1, 'D')).astype(int)
        
        ### Calculate adjustment for modeled fsnow
        df['fsnow_mod_adj'] = (df['SMB_at_SLA_obs_mwe'] 
                                / (df['PDD_cumsum_at_SLA_C'] 
                                    * df['days_since_melt_season_start_date']))
                
        ### Add adjustment to fsnow_mod
        df['fsnow_obs'] = fsnow_mod_site + df['fsnow_mod_adj']
        
        # Remove unrealistic values
        df.loc[df['fsnow_obs'] > 0.01, 'fsnow_obs'] = np.nan
        df.loc[df['fsnow_obs'] <= 0.0, 'fsnow_obs'] = np.nan
        
        ### Save the median
        fsnow_df = pd.DataFrame({'RGIId': [rgi_id],
                                 'fsnow_obs_m/C/d': [np.nanmedian(df['fsnow_obs'])],
                                 'fsnow_mod_m/C/d': [fsnow_mod_site]})
        fsnow_merged = pd.concat([fsnow_merged, fsnow_df], axis=0)
        
        ### Plot an example
        if rgi_id=='RGI60-01.00032':
            plt.rcParams.update({'font.size': 12, 'font.sans-serif': 'Arial'})
            fig, ax = plt.subplots(3, 1, figsize=(8,10))
            ax[0].plot(era_ds.time.data, era_ds['temp_C'], '-k', linewidth=0.5)
            ax[0].set_ylabel('Air temperature [$^{\circ}$C]')
            ax[0].grid()
            cmap = matplotlib.colors.LinearSegmentedColormap.from_list('my_cmap', ['w', '#fb6a4a', '#67000d']) # white to red
            era_ds['PDD_cumsum'].transpose().plot(cmap=cmap, ax=ax[1], 
                                                cbar_kwargs={'orientation': 'horizontal', 
                                                            'shrink': 0.5, 
                                                            'label': 'Cumulative PDD [$^{\circ}$C]'})
            ax[1].set_ylabel('Elevation [m]')
            ax[1].plot(slas_site.time.values, slas_site['SLA_obs_m'].values, '*k', label='Snowline altitude')
            # melt season start
            for year in era_ds.year.data[1:]:
                era_ds_year = era_ds.sel(time=slice(f"{year}-01-01", f"{year}-08-01"))
                xmesh, ymesh = np.meshgrid(era_ds_year.time.data, era_ds.h.data)
                ax[1].contour(xmesh, ymesh, era_ds_year['PDD_cumsum'].data.transpose(), levels=[0], colors=['gray'])
            ax[1].plot(pd.Timestamp('2010-01-01'), 0, '-', color='gray', label='Melt season start')
                        
            ax[1].legend(loc='upper left')
            ax[1].set_xlabel('')
            ax[2].plot(df['Date'], df['fsnow_obs'] * 1e3, '.k')
            ax[2].axhline(fsnow_merged['fsnow_mod_m/C/d'].values[0] * 1e3, color='k', linestyle='--', label='Modeled')
            ax[2].axhline(fsnow_merged['fsnow_obs_m/C/d'].values[0] * 1e3, color='k', label='Observed median')
            ax[2].legend(loc='lower left')
            ax[2].set_ylabel('$f_{snow}$ [mm $^{\circ}$C$^{-1}$ d$^{-1}$]')
            ax[2].grid()
            ax[2].set_ylim(2, 5)
            labels = ['a', 'b', 'c']
            for i, axis in enumerate(ax):
                axis.set_xlim(np.datetime64('2013-01-01'), np.datetime64('2023-01-01'))
                axis.text(0.97, 0.85, labels[i], transform=axis.transAxes, fontweight='bold', fontsize=16)
            fig.tight_layout()
            plt.show()
            # save figure
            fig_fn = os.path.join(code_path, 'figures', 'figS5_melt_factors_example.png')
            fig.savefig(fig_fn)
            print('Figure saved to file:', fig_fn)
        
    # Save results to file
    fsnow_merged.to_csv(fsnow_merged_fn, index=False)
    print('Observed fsnow saved to file:', fsnow_merged_fn)  
    
else:
    fsnow_merged = pd.read_csv(fsnow_merged_fn)
    print('Observed fsnow loaded from file')  

plt.hist(fsnow_merged['fsnow_obs_m/C/d'] * 1e3, bins=100)
plt.xlabel('Melt factor of snow [mm $^{\circ}$C$^{-1}$ d$^{-1}$]')
plt.ylabel('Counts')
plt.show()    

In [ ]:
# Plot differences between modeled and observed fsnow
import seaborn as sns
fsnow_merged['fsnow_mod-fsnow_obs_m/C/d'] = fsnow_merged['fsnow_mod_m/C/d'] - fsnow_merged['fsnow_obs_m/C/d']
fsnow_merged['fsnow_mod-fsnow_obs_mm/C/d'] = fsnow_merged['fsnow_mod-fsnow_obs_m/C/d'] * 1e3

clusters_fn = os.path.join(out_path, 'climate_clusters.csv')
clusters = pd.read_csv(clusters_fn)

fsnow_merged = pd.merge(fsnow_merged, clusters[['RGIId', 'clustName']], on='RGIId')

sns.boxplot(fsnow_merged, x='fsnow_obs_m/C/d', hue='clustName')
plt.show()
sns.kdeplot(fsnow_merged, x='fsnow_mod-fsnow_obs_mm/C/d', hue='clustName') #, clip=(-0.5, 0.5))
plt.show() 